So far all we've looked at have been static problems. Now we will do something *time dependent*, as a motivator to explore more of Julia's linear algebra and parallel features.

We will start in a situation with non-zero transverse magnetic field and turn it off, seeing what happens. This is going to require doing *a lot* of independent matrix-vector operations - a good use case for parallelism!

First, we'll make our Hamiltonian from the previous part, with a little time-dependent spice:

$$ \hat{H}(t) = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^z \hat{\sigma}_j^z - h(t)\sum_i \hat{\sigma}_i^x $$

Now $h(t)$ is some time dependent function. This means that the lowest energy state (the groundstate) will change with time as well. We need a way to simulate this. In a closed quantum system, a wavefunction $|\Psi\rangle$ will undergo *unitary time evolution* so that:

$$ \left| \Psi(t) \right\rangle = \hat{U}(t)\left| \Psi(t = 0)\right\rangle $$

so that

$$ \left| \left\langle \Psi(t) \left|\right. \Psi(t) \right\rangle\right|_2 = 1 \forall t $$

$\hat{U}(t)$ is a *unitary* operator - it's norm preserving. The particular form it takes is:

$$ \hat{U}(t) = \exp\left\{- i t\hat{H}(t) \right\} $$

Since $\hat{H}(t)$ is always Hermitian, $\hat{U}(t)$ is always unitary. For an explanation of where all this comes from you can consult a textbook on quantum mechanics. For now, if it's confusing, we're just going to
  1. Calculate $\hat{U}(t)$ for various times (and perhaps use a few shortcuts)
  2. Multiply it by $|\Psi\rangle$ to find the groundstate at various times
  3. Make pretty pictures, learn some things
  
Remember that $|\Psi\rangle$ is "just" some vector, and $\hat{U}$ and $\hat{H}$ are matrices - underneath all the jargon, we're still just doing linear algebra! We'll be able to reuse our types from the previous part:

Now we can write some single-node code to generate the wavefunction at various times, which we'll parallelize in a moment. It's good to have a working single-node version as a proof of concept and something to test against.

In [ ]:
function timeEvolve(psi, H, t)
    U = expm(-im*t*Hermitian(H)) # want to use the optimized method
    return U*psi
end

function timeSeries(psi, H, start, step, stop)
    times = linspace(start, step, stop)
    map((t,)->timeEvolve(t, psi, H), times)
end

## Here's your parallelism, dude

Now we can use Julia's [parallel map](https://docs.julialang.org/en/latest/stdlib/parallel.html#Base.Distributed.pmap) function to make this faster! For now, all we have to do is add the "`p`": 

In [ ]:
function timeSeriesParallel(psi, H, start, step, stop)
    times = linspace(start, step, stop)
    pmap((t,)->timeEvolve(t, psi, H), times)
end

Oops! The workers don't know about our type and other functions. We'll need to load them onto each worker to be able to use `pmap`. For a more detailed discussion of why this is, consult the [docs](https://docs.julialang.org/en/latest/manual/parallel-computing.html#Code-Availability-and-Loading-Packages-1).

Vroom vroom! Look at that nice little speedup. This is an *embarrassingly parallel* problem. How convenient that the workers don't need to coordinate with each other, only with the driver node. Now we can make an initial plot:

There are clearly a lot of inefficiencies in this code. Let's enumerate some of them:
  1. We are constructing each Hamiltonian matrix on the head node and sending the whole thing to the workers. All the workers need is the value of $h(t)$.
  2. We are sending the entire groundstate back when, for now, all we need is the magnetization (a single `float`).
  3. The Hamiltonian is actually diagonal in the basis we have picked. We can use a linear algebra trick to speed up the hard work of computing $\hat{U}(t)$.